# EMagPy Quickstart
In this tutorial you will learn how to import and processes EMI data with the emagpy API (Application Progamming Interface). The API is a set of classes, function and methods that can be called from the command line or in script and jupyter notebook. EMagPy also has a graphical user interface (GUI) that make use of the API behind the scene. The advantage of the API is that is allows more automated task to be performed and user in interactive environment such as this jupyter notebook.

## Import EMagPy
You can either download emagpy from source from https://gitlab.com/hkex/emagpy or install it using `pip install emagpy`. Then you can import the main class from the emagpy module: `Problem`.

In [1]:
import os
import numpy as np
import sys
import pandas as pd
sys.path.insert(0,'../src/') # this add the emagpy/src directory to the PATH
from emagpy import Problem # import the main Problem class from emagpy

## Data import
After importing the module we will import the file `coverCrop.csv` from the test folder.

In [2]:
testdir = '../src/examples/cover-crop/'

k = Problem() # this create the main object
k.createSurvey(testdir + 'coverCrop.csv') # this import the data

The data are usually imported from a .csv file with the headers being the coil configuration. Below is an example of the `coverCrop.csv` file.

In [3]:
import pandas as pd
df = pd.read_csv(testdir + 'coverCrop.csv')
df.head() # see how the header are formatted

## Data visualization
You can use the `Problem.show()` function to visualize the data as a line graph. Additionnaly if spatial coordinates are in the csv file (columns x an y). A map can also be produced using `Problem.showMap()`.

In [4]:
k.show(vmax=50)

In [5]:
k.showMap(coil='VCP0.71', contour=True, pts=True)

## Data inversion
To invert the data we must first define a starting model with a given number of layers and a starting conductivity in mS/m.

In [6]:
k.setInit(depths0=[0.5, 1], # specify the BOTTOM of each layer (the last layer is infinite)
          conds0=[20, 20, 20],
          fixedConds=[False, False, False])# conductivity in mS/m

Then there are different forward model available:
- `CS` : Cumulative Senstivity (following McNeil 1980). Default and fastest forward model.
- `CSgn` : Which is a fast Gauss-Newton inversion based on CS. Choosing this forward model will automatically make use of the `Problem.ìnvertGN()` method and ignore the arguments passed to `invert()` except `alpha`.
- `FSlin` : Full Solution based on Maxwell's equations using the LIN (Low Induction Number) approximation to convert the quadrature (Q) to ECa
- `FSeq` : Full Solution based on Maxwell's equations that doesn't rely on the LIN but rather compute an apparent value based on optimization (see Andrade et al., 2016).
- `Q` : Full Solution based on Maxwell's equations where the quadrature (Q) is directly minimize without being converted to ECa. This forward model is prefered for high EC (> 100 mS/m).

To solve the optimization problem, two types of solvers are available:
- `L-BFGS-B` (default), `TNC`, `CG`, `Nelder-Mead` minimizes an objective function using `scipy.optimize.minimize()`.
- `ROPE`, `SCEUA`, `DREAM` are MCMC-based method provided by the `spotpy` package that works well when the depth of each layer is not fixed.

The inverted section can be seen using `showResults()` and `showSlice()` if spatial data are available. The quality of the inversion can be assessed using `showMisfit()` and `showOne2one()`.

In [7]:
# default minimize() based inversion with CS and L-BFGS-B solver
k.invert()
k.showResults()
k.showMisfit()
k.showOne2one()

In [8]:
k.showSlice(islice=0, contour=True, vmin=12, vmax=50)
k.showSlice(islice=2, contour=True, vmin=12, vmax=50)

In [9]:
help(k.showSlice) # how to access the help for each method